In [55]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import keras
import numpy as np
from tqdm import tqdm

from music21 import stream as music21_stream
from music21 import converter as music21_converter
from music21 import instrument as music21_instrument
from music21 import note as music21_note
from music21 import chord as music21_chord
from glob import glob

In [45]:
def get_notes(midi_dir):
    notes = []
    for fname in glob(os.path.join(midi_dir, '*.mid')):
        print('Parsing', fname)
        midi = music21_converter.parse(fname)
        
        notes_to_parse = None
        try: # file has instrument part
            s2 = music21_instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes            
        
        for element in notes_to_parse:
            if isinstance(element, music21_note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21_chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        
    return notes_to_parse, notes

In [46]:
midi_dir = os.path.join(os.getcwd(), 'Classical-Piano-Composer/data')
notes_to_parse, notes = get_notes(midi_dir)

Parsing /content/dohai90/workspace/keras/Classical-Piano-Composer/data/8-Copy1.mid


In [47]:
notes

['C5',
 'E4',
 'C2',
 'C5',
 'E4',
 'C2',
 'G4',
 'G4',
 'C5',
 'C5',
 'D5',
 '10.2',
 'B-1',
 'D5',
 '10.2',
 'B-1',
 'F4',
 'F4',
 'B-4',
 'B-4',
 'F5',
 '9.0',
 'F1',
 'F5',
 '9.0',
 'F1',
 'A5',
 '9.0',
 'F1',
 'A5',
 '9.0',
 'F1',
 'G5',
 '11.2',
 'G1',
 'G5',
 '11.2',
 'G1',
 'E5',
 'C5',
 'C2',
 'E5',
 'C5',
 'C2',
 'C2',
 'C2',
 'G4',
 'G4',
 'C5',
 'C5',
 'D5',
 'G1',
 'D5',
 'G1',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G1',
 'G1',
 'G4',
 'C5',
 'C2',
 'G4',
 'C5',
 'C2',
 'C2',
 'C2',
 'A3',
 'G4',
 'A3',
 'G4',
 'C4',
 'C5',
 'C4',
 'C5',
 'B-3',
 'B-4',
 'G1',
 'B-3',
 'B-4',
 'G1',
 'D4',
 'B-4',
 'D4',
 'B-4',
 'C4',
 'C5',
 'C4',
 'C5',
 'F4',
 'D5',
 'F4',
 'D5',
 'G1',
 'G1',
 'G4',
 'E5',
 'C2',
 'G4',
 'E5',
 'C2',
 'C2',
 'C2',
 'G4',
 'D5',
 'G4',
 'D5',
 'A4',
 'E5',
 'A4',
 'E5',
 'A4',
 'F5',
 'G1',
 'A4',
 'F5',
 'G1',
 'F4',
 'D5',
 'F4',
 'D5',
 'F4',
 'D5',
 'F4',
 'D5',
 'B-4',
 'B-5',
 'G1',
 'B-4',
 'B-5',
 'G1',
 'B-4',
 'F5',
 'B

In [4]:
maxlen = 100
step = 1
sequences = []
next_notes = []

for i in range(0, len(notes) - maxlen, step):
    sequences.append(notes[i:i+maxlen])
    next_notes.append(notes[i+maxlen])

print('Number of sequences', len(sequences))

labels = sorted(list(set(notes)))
print('Number of labels', len(labels))

note_indices = dict((note, i) for i, note in enumerate(labels))

print('Vectorizing...')
x = np.zeros((len(sequences), maxlen, len(labels)))
y = np.zeros((len(sequences), len(labels)))
for i, sequence in enumerate(sequences):
    for t, note in enumerate(sequence):
        x[i, t, note_indices[note]] = 1
    y[i, note_indices[next_notes[i]]] = 1

Number of sequences 57077
Number of labels 358
Vectorizing...


In [6]:
# building model
model = keras.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(maxlen, len(labels))))
model.add(keras.layers.Dense(len(labels), activation=keras.activations.softmax))

model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

fp = '/content/dohai90/workspace/keras/checkpoints/melody_gen_{epoch:02d}.h5'
callbacks_list = [keras.callbacks.ModelCheckpoint(filepath=fp,
                                                  monitor='loss',
                                                  save_best_only=True),
                  keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy',
                                                patience=5)]

history = model.fit(x, y, epochs=200, batch_size=128, validation_split=0.2, callbacks=callbacks_list)

Train on 45661 samples, validate on 11416 samples
Epoch 1/200
45661/45661 [==============================] - 138s 3ms/step - loss: 3.9557 - categorical_accuracy: 0.1310 - val_loss: 4.0947 - val_categorical_accuracy: 0.1258
Epoch 2/200
45661/45661 [==============================] - 138s 3ms/step - loss: 3.0940 - categorical_accuracy: 0.2585 - val_loss: 4.0977 - val_categorical_accuracy: 0.1451
Epoch 3/200
45661/45661 [==============================] - 136s 3ms/step - loss: 2.6260 - categorical_accuracy: 0.3626 - val_loss: 4.2184 - val_categorical_accuracy: 0.1427
Epoch 4/200
45661/45661 [==============================] - 138s 3ms/step - loss: 2.2663 - categorical_accuracy: 0.4506 - val_loss: 4.3438 - val_categorical_accuracy: 0.1390
Epoch 5/200
45661/45661 [==============================] - 136s 3ms/step - loss: 1.9858 - categorical_accuracy: 0.5192 - val_loss: 4.4561 - val_categorical_accuracy: 0.1422
Epoch 6/200
45661/45661 [==============================] - 138s 3ms/step - loss: 1.76

In [41]:
model.load_weights('/content/dohai90/workspace/keras/checkpoints/melody_gen_02.h5')

In [50]:
def create_midi(prediction_output, fp):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    midi_output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = music21_note.Note(int(current_note))
                new_note.storedInstrument = music21_instrument.Piano()
                notes.append(new_note)
            new_chord = music21_chord.Chord(notes)
            new_chord.offset = offset
            midi_output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = music21_note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = music21_instrument.Piano()
            midi_output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = music21_stream.Stream(midi_output_notes)

    midi_stream.write('midi', fp=fp)

In [51]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [56]:
start_index = 0
seed_sequence = notes[start_index:start_index + maxlen]

for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('[INFO] temperature', temperature)
    output_notes = seed_sequence[:]
    generated_notes = seed_sequence[:]
    
    for i in tqdm(range(400)):
        sampled = np.zeros((1, maxlen, len(labels)))
        for t, note in enumerate(generated_notes):
            sampled[0, t, note_indices[note]] = 1.
            
        preds = model.predict(sampled)[0]
        next_index = sample(preds, temperature)
        next_note = labels[next_index]
        
        output_notes.append(next_note)
        generated_notes = output_notes[i+1:i+1+maxlen]
    
    fp = 'test_output_{}.mid'.format(temperature)
    create_midi(output_notes, fp)        

  0%|          | 2/400 [00:00<00:30, 12.93it/s]

100
[INFO] temperature 0.2
output_notes 100
generated_notes 100


  0%|          | 2/400 [00:00<00:24, 16.42it/s]

[INFO] temperature 0.5
output_notes 100
generated_notes 100


  0%|          | 2/400 [00:00<00:23, 17.05it/s]

[INFO] temperature 1.0
output_notes 100
generated_notes 100


  0%|          | 2/400 [00:00<00:21, 18.29it/s]

[INFO] temperature 1.2
output_notes 100
generated_notes 100


100%|██████████| 400/400 [00:24<00:00, 16.02it/s]
